# README

Please download the model `DQN_model.keras`  from the Kaggle.

Model Link: https://www.kaggle.com/models/weichihsu1996/dqn-model-on-car-racing-v3-random-environment/

Then upload the model to **Colab**.

Then **RUN** all cells.

Note: If you don't want to create the model, you can skip the `build_model` cell. I leave the cell here for users who want to see source code of model. `

In [ ]:
# Instal Gym
! pip install gymnasium
! pip install swig
! pip install gymnasium[box2d]
# Install OpenCV
! pip install opencv-python

# ==== Note ====
# If you are run it in local jupyter lab, uncomment this line
!pip install box2d pygame

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp311-cp311-linux_x86_64.whl size=2379367 sha256=b7b6e792acdcb02115b4871d588060eb71068a2de9b95853b4ea328e71534176
  Stored in directory: /root/.cache/pip/wheels/ab/f1/0c/d56f4a2bdd12bae0a0693ec33f2f0daadb5eb9753c78fa5308
Successfully built box2d-py
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 44.1 MB/s eta 0:00:00


In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
from collections import deque
from tensorflow.keras.models import Model

# ==== Layers ====
from tensorflow.keras.layers import Input, InputLayer, Conv2D, Flatten, Dense, Reshape, Conv2DTranspose, LeakyReLU, Dropout, BatchNormalization, ReLU, MaxPooling2D, UpSampling2D, Concatenate, Activation, Lambda, Add, LayerNormalization, LeakyReLU, AveragePooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, ZeroPadding2D, Cropping2D, ZeroPadding1D, Cropping1D, Layer, Average, AveragePooling1D, GlobalAveragePooling1D, GRU

from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization

# ==== Optimizer ====
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl

# ==== Callbacks ====
from tensorflow.keras.callbacks import EarlyStopping

# ==== Saving ====
from keras.saving import register_keras_serializable

# ==== Get File ====
from tensorflow.keras.utils import get_file

# ==== tqdm ====
from tqdm import tqdm
from tqdm import trange

# ==== Counter ====
from collections import Counter

# ==== Statistics ====
import scipy.stats

# ==== Video ====
import imageio
import os
import cv2 # Import cv2 here

In [ ]:
# ==== Nonlinear Contrast Enhancement ====

# ==== Register your layer ====
@register_keras_serializable()

class ContrastEnhance(Layer):
    def __init__(self, alpha=1.5, **kwargs):
        super(ContrastEnhance, self).__init__(**kwargs)
        self.alpha = alpha

    def call(self, x):
        mean = tf.reduce_mean(x, axis=[1, 2], keepdims=True)
        return tf.clip_by_value(self.alpha * (x - mean) + mean, 0.0, 1.0)

In [ ]:
# ==== Sober ====

# ==== Register your layer ====
@register_keras_serializable()

class SobelEdgeLayer(Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # Sobel kernels (3x3)
        self.sobel_x = tf.constant([[[-1, 0, 1],
                                     [-2, 0, 2],
                                     [-1, 0, 1]]], dtype=tf.float32)
        self.sobel_y = tf.constant([[[-1, -2, -1],
                                     [ 0,  0,  0],
                                     [ 1,  2,  1]]], dtype=tf.float32)
        self.sobel_45 = tf.constant([[[-2, -1, 0],
                              [-1,  0, 1],
                              [ 0,  1, 2]]], dtype=tf.float32)

        self.sobel_135 = tf.constant([[[ 0, -1, -2],
                               [ 1,  0, -1],
                               [ 2,  1,  0]]], dtype=tf.float32)

    def call(self, x):
        # Input shape: (batch, H, W, C)
        channels = x.shape[-1]
        kernel_x = tf.expand_dims(self.sobel_x, axis=-1)
        kernel_y = tf.expand_dims(self.sobel_y, axis=-1)

        # Repeat for depthwise convolution
        kernel_x = tf.repeat(kernel_x, repeats=channels, axis=-1)
        kernel_y = tf.repeat(kernel_y, repeats=channels, axis=-1)

        kernel_x = tf.reshape(kernel_x, [3, 3, channels, 1])
        kernel_y = tf.reshape(kernel_y, [3, 3, channels, 1])

        # Apply Sobel filters
        gx = tf.nn.depthwise_conv2d(x, kernel_x, strides=[1, 1, 1, 1], padding="SAME")
        gy = tf.nn.depthwise_conv2d(x, kernel_y, strides=[1, 1, 1, 1], padding="SAME")

        grad_magnitude = tf.sqrt(tf.square(gx) + tf.square(gy) + 1e-6)  # prevent sqrt(0)

        # Test
        #x = 0.2*(1-grad_magnitude)

        return grad_magnitude

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras.initializers import Constant
import numpy as np

def create_gabor_filter_bank(num_filters, ksize):
    filters = []
    for i in range(num_filters):
        theta = i * np.pi / num_filters  # orientation
        sigma = 2.0
        Lambda = 5.0
        psi = 0
        gamma = 0.5

        # Gabor kernel formula
        xmax = ksize // 2
        ymax = ksize // 2
        xmin = -xmax
        ymin = -ymax
        (y, x) = np.meshgrid(np.linspace(ymin, ymax, ksize), np.linspace(xmin, xmax, ksize))
        x_theta = x * np.cos(theta) + y * np.sin(theta)
        y_theta = -x * np.sin(theta) + y * np.cos(theta)
        gb = np.exp(-0.5 * (x_theta**2 + gamma**2 * y_theta**2) / sigma**2) * np.cos(2 * np.pi * x_theta / Lambda + psi)

        filters.append(gb)

    filters = np.array(filters)  # (num_filters, ksize, ksize)
    filters = filters[:, :, :, np.newaxis]  # (num_filters, ksize, ksize, 1)
    filters = np.transpose(filters, (1, 2, 3, 0))  # (ksize, ksize, 1, num_filters)
    return filters.astype(np.float32)

# ==== Register your layer ====
@register_keras_serializable()
class GaborConv2D(Layer):
    def __init__(self, filters=8, ksize=11, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.ksize = ksize

    def build(self, input_shape):
        in_channels = input_shape[-1]
        gabor = create_gabor_filter_bank(self.filters, self.ksize)  # shape (ksize, ksize, 1, filters)
        gabor = np.tile(gabor, (1, 1, in_channels, 1))  # (ksize, ksize, in_channels, filters)
        self.gabor_kernel = self.add_weight(
            shape=gabor.shape,
            initializer=Constant(gabor),
            trainable=False,
            name='gabor_filter'
        )

    def call(self, inputs):
        return tf.nn.conv2d(inputs, self.gabor_kernel, strides=1, padding='SAME')

In [ ]:
@register_keras_serializable()
class VisualEnhanceBlock(Layer):
    def __init__(self, gene_code=None, **kwargs):
        super().__init__(**kwargs)

        # ==== Defaul Gene ====
        if gene_code is None:
            gene_code = [2, 0, 2.0, False]  # [contrast_times, sobel_times, alpha, use_residual]

        # ==== unpack ====
        self.contrast_times = int(gene_code[0])
        self.sobel_times = int(gene_code[1])
        self.alpha = float(gene_code[2])
        self.use_residual = bool(gene_code[3])

        # ==== layer list ====
        self.contrast_layers = [ContrastEnhance(alpha=self.alpha) for _ in range(self.contrast_times)]
        self.sobel_layers = [SobelEdgeLayer() for _ in range(self.sobel_times)]

    def call(self, inputs):
        x = inputs
        for contrast in self.contrast_layers:
            x = contrast(x)

        for sobel in self.sobel_layers:
            x = sobel(x) + x  # Enhance edge

        if self.use_residual:
            return Add()([x, inputs])
        else:
            return x

    def get_config(self):
        config = super().get_config()
        config.update({
            "gene_code": [self.contrast_times, self.sobel_times, self.alpha, self.use_residual]
        })
        return config

In [ ]:
# ==== Dueling Q Output ====

# ==== Register your layer ====
@register_keras_serializable()
class DuelingQOutput(tf.keras.layers.Layer):
    def call(self, inputs):
        v, a = inputs
        return v + (a - tf.reduce_mean(a, axis=1, keepdims=True))

@register_keras_serializable()
def dueling_q_output(x):
    v, a = x
    return v + (a - tf.reduce_mean(a, axis=1, keepdims=True))


# ==== How to USE ====
# q = DuelingQOutput()(x)

In [ ]:
def res_block(x, filters):
    shortcut = x
    x = Conv2D(filters, 3, padding='same', activation=None)(x)
    x = ReLU()(x)
    x = Conv2D(filters, 3, padding='same', activation=None)(x)
    x = Add()([x, shortcut])
    x = ReLU()(x)
    return x

In [ ]:
# === Type-0 Q-head ===
def build_q_head_type0(x, action_size = 5, head_id="1"):
    # Advantage Stream
    a = Dense(256, name=f"a_{head_id}_dense1")(x)
    a = Dense(action_size, name=f"a_{head_id}_out")(a)

    # Value Stream
    v = Dense(256, activation='relu', name=f"v_{head_id}_dense1")(x)
    v = Dense(1, name=f"v_{head_id}_out")(v)

    # Combine into Q-values
    q = DuelingQOutput()([v, a])
    return q

In [ ]:
# === Type-2 Q-head ===
def build_q_head_type2(x, action_size = 5, head_id="1"):
    # Advantage Stream
    a = Dense(256, name=f"a_{head_id}_dense1")(x)
    a = Dense(128, activation='relu', name=f"a_{head_id}_dense2")(a)
    a = Dense(action_size, name=f"a_{head_id}_out")(a)

    # Value Stream
    v = Dense(256, activation='relu', name=f"v_{head_id}_dense1")(x)
    v = Dense(128, name=f"v_{head_id}_dense2")(v)
    v = Dropout(0.2)(v)
    v = Dense(1, name=f"v_{head_id}_out")(v)

    # Combine into Q-values
    q = DuelingQOutput()([v, a])
    return q

In [ ]:
# === Type-5 Q-head ===
def build_q_head_type5(x, action_size = 5, head_id="1"):
    # Advantage Stream
    a = Dense(256, name=f"a_{head_id}_dense1")(x)
    a = Dense(128, activation='relu', name=f"a_{head_id}_dense2")(a)
    a = Dense(action_size, name=f"a_{head_id}_out")(a)

    # Value Stream
    v = Dense(256, activation='relu', name=f"v_{head_id}_dense1")(x)
    v = Dropout(0.2)(v)
    v = Dense(128, name=f"v_{head_id}_dense2")(v)
    v = Dense(1, name=f"v_{head_id}_out")(v)

    # Combine into Q-values
    q = DuelingQOutput()([v, a])
    return q

In [ ]:
def build_ensemble_heads(x, action_size, head_configs):
    head_builders = {
        "type0": build_q_head_type0,
        "type1": build_q_head_type1,
        "type2": build_q_head_type2,
        "type3": build_q_head_type3,
        "type4": build_q_head_type4,
        "type5": build_q_head_type5,
    }

    q_heads = []
    for i, head_type in enumerate(head_configs):
        if head_type is None:
            continue  # skip this head

        head_id = str(i + 1)
        if head_type == "type0":
            q = build_q_head_type0(x, action_size, head_id)
        elif head_type == "type1":
            q = build_q_head_type1(x, action_size, head_id)
        elif head_type == "type2":
            q = build_q_head_type2(x, action_size, head_id)
        elif head_type == "type3":
            q = build_q_head_type3(x, action_size, head_id)
        elif head_type == "type4":
            q = build_q_head_type4(x, action_size, head_id)
        elif head_type == "type5":
            q = build_q_head_type5(x, action_size, head_id)
        else:
            raise ValueError(f"Unknown head type: {head_type}")

        q_heads.append(q)

    return q_heads

In [ ]:
def build_conv_backbone(inputs):

  x = inputs

  # ==== Convolution Layers ====
  x = Conv2D(16, (8, 8), strides=4, activation = None)(x) # (23, 23, 16)
  x = ReLU()(x) # You can try ReLU or LeakyReLU

  # ==== Residual 1 ====

  x = res_block(x, 16) # (23, 23, 16)

  # Input 23 → (23 - 4)/2 + 1 = 10
  x = Conv2D(32, (4, 4), strides=2, activation = None)(x)
  x = ReLU()(x)

  # ==== Residual 2 ====
  x = res_block(x, 32) # (10, 10, 32)

  # Input 10 → (10 - 3)/1 + 1 = 8
  x = Conv2D(64, (3, 3), strides=1, activation = None)(x)
  x = ReLU()(x)

  # Input 8 → (8 - 3)/1 + 1 = 6
  x = Conv2D(128, (3, 3), strides=1, activation = None)(x)
  x = ReLU()(x)

  # ==== Flaten ====
  x = Flatten()(x)
  x = Dense(1024, activation='relu')(x)

  print("🔥 Conv2D layer created:", x)

  return x

In [ ]:
@register_keras_serializable()
class ConvBackbone(Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, inputs):
        x = Conv2D(16, (8, 8), strides=4)(inputs)
        x = ReLU()(x)

        # ==== Residual Block 1 ====
        x = self._res_block(x, 16)

        x = Conv2D(32, (4, 4), strides=2)(x)
        x = ReLU()(x)

        # ==== Residual Block 2 ====
        x = self._res_block(x, 32)

        x = Conv2D(64, (3, 3), strides=1)(x)
        x = ReLU()(x)

        x = Conv2D(128, (3, 3), strides=1)(x)
        x = ReLU()(x)

        x = Flatten()(x)
        x = Dense(1024, activation='relu')(x)
        return x

    def _res_block(self, x, filters):
        skip = x
        x = Conv2D(filters, (3, 3), padding='same')(x)
        x = ReLU()(x)
        x = Conv2D(filters, (3, 3), padding='same')(x)
        x = Add()([x, skip])
        x = ReLU()(x)
        return x

In [ ]:
def build_dense_mlp(x, units_list, activation='relu', name_prefix="mlp"):
    for idx, units in enumerate(units_list):
        if units is None or units == 0:
            continue  # skip this layer
        x = Dense(units, activation=activation, name=f"{name_prefix}_dense{idx+1}")(x)
    return x

In [ ]:
from tensorflow.keras.layers import Layer, Concatenate, Reshape
@tf.keras.utils.register_keras_serializable()
class SafeStack(Layer):
    def __init__(self, axis=1, **kwargs):
        super().__init__(**kwargs)
        self.axis = axis  # axis=1 equal to tf.stack(..., axis=1)

    def call(self, inputs):
        # inputs: list of tensors, shape = (batch_size, feature_dim)
        num_inputs = len(inputs)
        feature_dim = inputs[0].shape[-1]

        # Concatenate → shape = (batch_size, num_inputs * feature_dim)
        x = Concatenate(axis=-1)(inputs)

        # Reshape → shape = (batch_size, num_inputs, feature_dim)
        x = Reshape((num_inputs, feature_dim))(x)
        return x

In [ ]:
@tf.keras.utils.register_keras_serializable()
class SafeAttentionSum(tf.keras.layers.Layer):
    def call(self, inputs):
        merged, weights = inputs  # merged: (batch, 5, dim), weights: (batch, 5, 1)
        return tf.reduce_sum(merged * weights, axis=1)  # (batch, dim)

In [ ]:
# === Type-4 Q-head ===
def build_q_head_type4(x, action_size = 5, head_id="1"):
    # Advantage Stream
    a = Dense(256, name=f"a_{head_id}_dense1")(x)
    a = Dropout(0.2)(a)
    a = Dense(128, activation='relu', name=f"a_{head_id}_dense2")(a)
    a = Dense(action_size, name=f"a_{head_id}_out")(a)

    # Value Stream
    v = Dense(256, activation='relu', name=f"v_{head_id}_dense1")(x)
    v = Dense(128, name=f"v_{head_id}_dense2")(v)
    v = Dense(1, name=f"v_{head_id}_out")(v)

    # Combine into Q-values
    q = DuelingQOutput()([v, a])
    return q

In [ ]:
# === Type-3 Q-head ===
def build_q_head_type3(x, action_size = 5, head_id="1"):
    # Advantage Stream
    a = Dense(256, name=f"a_{head_id}_dense1")(x)
    a = Dense(128, activation='relu', name=f"a_{head_id}_dense2")(a)
    a = Dropout(0.2)(a)
    a = Dense(action_size, name=f"a_{head_id}_out")(a)

    # Value Stream
    v = Dense(256, activation='relu', name=f"v_{head_id}_dense1")(x)
    v = Dense(128, name=f"v_{head_id}_dense2")(v)
    v = Dense(1, name=f"v_{head_id}_out")(v)

    # Combine into Q-values
    q = DuelingQOutput()([v, a])
    return q

In [ ]:
# === Type-1 Q-head ===
def build_q_head_type1(x, action_size = 5, head_id="1"):
    # Advantage Stream
    a = Dense(256, name=f"a_{head_id}_dense1")(x)
    a = Dense(128, activation='relu', name=f"a_{head_id}_dense2")(a)
    a = Dense(action_size, name=f"a_{head_id}_out")(a)

    # Value Stream
    v = Dense(256, activation='relu', name=f"v_{head_id}_dense1")(x)
    v = Dense(128, name=f"v_{head_id}_dense2")(v)
    v = Dense(1, name=f"v_{head_id}_out")(v)

    # Combine into Q-values
    q = DuelingQOutput()([v, a])
    return q

In [ ]:
# ==== Build Model ====
# If you use stack frame, the input is (96, 96, 12)
# If you don't use stack frame, the input is (96, 96, 3)
def build_model (input_shape = (96, 96, 12), action_size = 5):
  inputs = Input(shape = input_shape) # (96, 96, 12)

  # ==== Normalization ====
  # Normalize pixel [0,255] → [0,1]
  x = tf.keras.layers.Rescaling(1./255)(inputs) # (96, 96, 12)

  # ==== Gray ====
  x = Conv2D(8, (1, 1), activation='relu')(x)  # (96, 96, 8)

  # ==== Visual Cortex ====

  # ==== Original ====
  # use raw preprocessed image (before contrast+sobel)
  x0 = x # (96, 96, 8)

  # ==== Contrast Enhancement ====

  # ==== x1 visual branch ===== Contrast Enhancement Only ===
  x1 = ContrastEnhance(alpha=2.0)(x0) # (96, 96, 8)
  x1 = ContrastEnhance(alpha=2.0)(x1) # (96, 96, 8)

  # ==== x2 visual branch ==== Sobel + Original Image
  x2 = SobelEdgeLayer()(x0)  # (96, 96, 8)
  x2 = Add()([x2, x0]) # (96, 96, 8)

  # ==== x3 visual branch ==== Contrast Enhancement + Sobel
  x3 = Add()([x1, x2]) # (96, 96, 8)

  # ==== x4 visual branch ==== Gabor Only
  x4 = 0.01*GaborConv2D(filters=8, ksize=11)(x0) + x0 # (96, 96, 8)

  # ==== Convolution ====

  # ==== Convolution Backbone  1====
  x0 = build_conv_backbone(x0) # (128,)

  # ==== Convolution Backbone  2====
  x1 = build_conv_backbone(x1) # (128,)

  # ==== Convolution Backbone  3====
  x2 = build_conv_backbone(x2) # (128,)

  # ==== Convolution Backbone 4 ====
  x3 = build_conv_backbone(x3) # (128,)

  # ==== Convolution Backbone 5 ====
  x4 = build_conv_backbone(x4) # (128,)

  # Define the embedding dimension for positional encoding and attention
  embedding_dim = 1024

  # Add a Dense layer to make the feature dimension match the embedding_dim
  x0 = Dense(embedding_dim, activation='relu')(x0) # (None, 128)
  x1 = Dense(embedding_dim, activation='relu')(x1) # (None, 128)
  x2 = Dense(embedding_dim, activation='relu')(x2) # (None, 128)
  x3 = Dense(embedding_dim, activation='relu')(x3) # (None, 128)
  x4 = Dense(embedding_dim, activation='relu')(x4) # (None, 128)

  # ==== Combine CNN 1 and 2 ====
  # === Merge branches ===
  # Method 1
  #merged = Add()([x0, x1, x2, x3, x4]) # (128,)

  # Method 2
  #merged = SafeStack()([x0, x1, x2, x3, x4]) # shape = (batch_size, 5, 128)
  #x = GlobalAveragePooling1D()(merged)

  # Method 3
  merged = SafeStack()([x0, x1, x2, x3, x4])  # (batch, 5, 128)
  weights = Dense(1, activation='softmax')(merged)  # (batch, 5, 1)
  x = SafeAttentionSum(name="attention_sum")([merged, weights]) # (batch, 128)

  # ==== Deep Layers ====
  # ✅ ==== Brain Gene Code ====
  brain_gene = [512, 256, 128, 64, None]

  x = build_dense_mlp(x, units_list= brain_gene, activation='relu', name_prefix= 'brain')

  # ==== Add Attention ====
  # Shall we try this?
  #x = Reshape((1, -1))(x)   # => shape = (batch, 1, dim) for attention
  #x = GRU(128, return_sequences=True)(x)
  # x = Bidirectional(GRU(64, return_sequences=True))(x)
  #x = MultiHeadAttention(num_heads=4, key_dim=32)(x, x)
  #x = GlobalAveragePooling1D()(x)

  # ==== Dueling DQN ==== Original Version ====
  # Two head mechanism
  # Advantage Stream
  #a = Dense(256, activation= 'relu')(x)
  #a = NoisyDense(128, sigma_init=0.05)(a)
  #a = Dense(action_size)(a)


  # Value Stream
  #v = Dense(256, activation = 'relu')(x)
  #v = Dense(128)(v)
  #v = Dense(1)(v)

  # ==== Ensemble Dueling DQN ====

  # ==== Use Head Builder ====

  # ✅ ==== Head Gene Code ===
  # Use type1 ~ type 5 head
  # let head_gene be [type1, type2, ..., type5]
  # The best one is just the head_gene = ['type1', 'type2', 'type3', 'type4', 'type5'] only
  head_gene = []
  for _ in range(2):
      head_gene.append( ['type1', 'type2', 'type3', 'type4', 'type5'])

  # Reshape head_gene  to be 1 dimension array
  head_gene = [item for sublist in head_gene for item in sublist]

  #print(head_gene)

  q_heads = build_ensemble_heads(x, action_size= 5, head_configs = head_gene)

  # ==== Combine ====

  q_mean = Average()(q_heads)

  # ==== Output ====
  # Q-values for each discrete action
  # Output: q_values = [[12.3, 11.8, 9.5, 14.2, 13.0]]
  # The q_values of each action
  # i.e., Q(s, a = 0) = 12.3
  # i.e., Q(s, a =1) = 11.8
  #outputs = Dense(action_size, activation='linear', name = 'Q-values')(x)

  # ==== Dueling DQN Output ====
  # Q(s, a) = V(s) + (A(s, a) - mean(A(s, a')))
  # V(s) -> (batch_size, 1)
  # A(s, a) -> (batch_size, action_size)

  # ==== Output ====
  # Note: Remember add "output_shape = (None, action_size)"
  #outputs = Lambda(dueling_q_output, output_shape=(action_size,))([v, a])

  outputs = q_mean

  # ==== Model Build ====
  model = Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer=Adam(learning_rate=0.00025), loss='huber')

  return model

# ==== Car Racing parameters ====
state_size = 96*96*12
action_size = 5

# ==== Structure ====
test_model = build_model(input_shape = (96, 96, 12), action_size = 5)
test_model.summary()
tf.keras.utils.plot_model(test_model, show_shapes=True, to_file="model_graph.png")

🔥 Conv2D layer created: <KerasTensor shape=(None, 1024), dtype=float32, sparse=False, name=keras_tensor_30>
🔥 Conv2D layer created: <KerasTensor shape=(None, 1024), dtype=float32, sparse=False, name=keras_tensor_50>
🔥 Conv2D layer created: <KerasTensor shape=(None, 1024), dtype=float32, sparse=False, name=keras_tensor_70>
🔥 Conv2D layer created: <KerasTensor shape=(None, 1024), dtype=float32, sparse=False, name=keras_tensor_90>
🔥 Conv2D layer created: <KerasTensor shape=(None, 1024), dtype=float32, sparse=False, name=keras_tensor_110>


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 96, 96,    │          0 │ -                 │
│ (InputLayer)        │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 96, 96,    │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 96, 96, 8) │        104 │ rescaling[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ contrast_enhance    │ (None, 96, 96, 8) │          0 │ conv2d[0][0]      │
│ (ContrastEnhance)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sobel_edge_layer    │ (None, 96, 96, 8) │          0 │ conv2d[0][0]      │
│ (SobelEdgeLayer)    │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gabor_conv2d        │ (None, 96, 96, 8) │      7,744 │ conv2d[0][0]      │
│ (GaborConv2D)       │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ contrast_enhance_1  │ (None, 96, 96, 8) │          0 │ contrast_enhance… │
│ (ContrastEnhance)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 96, 96, 8) │          0 │ sobel_edge_layer… │
│                     │                   │            │ conv2d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 96, 96, 8) │          0 │ gabor_conv2d[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 96, 96, 8) │          0 │ contrast_enhance… │
│                     │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 96, 96, 8) │          0 │ multiply[0][0],   │
│                     │                   │            │ conv2d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 23, 23,    │      8,208 │ conv2d[0][0]      │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 23, 23,    │      8,208 │ contrast_enhance… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_17 (Conv2D)  │ (None, 23, 23,    │      8,208 │ add[0][0]         │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_25 (Conv2D)  │ (None, 23, 23,    │      8,208 │ add_1[0][0]       │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_33 (Conv2D)  │ (None, 23, 23,    │      8,208 │ add_2[0][0]       │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 23, 23,    │          0 │ conv2d_1[0][0]    │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 31,210,293 (119.06 MB)

 Trainable params: 31,202,549 (119.03 MB)

 Non-trainable params: 7,744 (30.25 KB)

In [ ]:
# ==== Frame Stack ====
# Initialize buffer
NUM_FRAMES = 4
frame_buffer = deque(maxlen=NUM_FRAMES)

def preprocess_frame(frame, size=(96, 96), grayscale=False):
    if grayscale:
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, size)
        frame = np.expand_dims(frame, axis=-1)  # (96,96,1)
    else:
        frame = cv2.resize(frame, size)  # RGB
    return frame.astype(np.uint8)

In [ ]:
from keras.layers import Layer
import tensorflow as tf
from keras.saving import register_keras_serializable



DQN_model = tf.keras.models.load_model('/content/DQN_model.keras')
DQN_model.summary()

# Re-initialize the environment for evaluation
# Use the same parameters as the training environment

# ==== GIF file ====
output_dir = "10_DQN_agnet_demo_gifs"  # Name your file
os.makedirs(output_dir, exist_ok=True)  # Create it

# ==== Evaluation ====
evaluation_episodes = 10 # Number of evaluation
score = []

max_step = 1000

# ==== Initialize Environment ====
env = gym.make("CarRacing-v3",
              render_mode= 'rgb_array',
              lap_complete_percent=0.95,
              domain_randomize=True,
              continuous=False)



for episode in range(evaluation_episodes):
  # ==== Make GIF ====
  gif_frames = []

  obs, _ = env.reset()
  frame = preprocess_frame(obs)

  # Initialize frame buffer
  frame_buffer = deque([frame]*NUM_FRAMES, maxlen=NUM_FRAMES)
  stacked_state = np.concatenate(list(frame_buffer), axis=-1)  # shape: (96, 96, 12) if RGB

  total_reward = 0


  for time in range(max_step):
    gif_frames.append(env.render())

    # Step 1: Select action
    q_values = DQN_model(stacked_state[np.newaxis, ...], training=True)[0]
    action = np.argmax(q_values)

    # Step 2: Interaction with env.
    next_obs, reward, terminated, truncated, _ = env.step(action)

    # Step 3: Preprocess frame
    next_frame = preprocess_frame(next_obs)

    # Step 4: Update frame buffer + stacked_state
    frame_buffer.append(next_frame)
    stacked_state = np.concatenate(list(frame_buffer), axis=-1)

    # Step 5: Update reward
    total_reward += float(reward)

    # Step 6: Check if done
    done = terminated or truncated
    if done:
        break

  score.append(total_reward)
  print(f"Episode: {episode+1}/{evaluation_episodes}, Score: {total_reward:.2f}")
  gif_path = os.path.join(output_dir, f"episode_{episode+1:03d}-{total_reward:.0f}.gif")
  imageio.mimsave(gif_path, gif_frames, fps=30)

env.close()

# ==== Scores ====
average_score = np.mean(score)
max_score = np.max(score)
min_score = np.min(score)
std_score = np.std(score)

print(f'Evaluation complete after {evaluation_episodes} Episodes')
print(f"Average Score: {average_score:.2f}")
print(f"Max Score: {max_score:.2f}")
print(f"Min Score: {min_score:.2f}")
print(f"Std Score: {std_score:.2f}")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 466 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 96, 96,    │          0 │ -                 │
│ (InputLayer)        │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_3         │ (None, 96, 96,    │          0 │ input_layer_3[0]… │
│ (Rescaling)         │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_123 (Conv2D) │ (None, 96, 96, 8) │        104 │ rescaling_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ contrast_enhance_16 │ (None, 96, 96, 8) │          0 │ conv2d_123[0][0]  │
│ (ContrastEnhance)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sobel_edge_layer_5  │ (None, 96, 96, 8) │          0 │ conv2d_123[0][0]  │
│ (SobelEdgeLayer)    │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gabor_conv2d_5      │ (None, 96, 96, 8) │      7,744 │ conv2d_123[0][0]  │
│ (GaborConv2D)       │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ contrast_enhance_17 │ (None, 96, 96, 8) │          0 │ contrast_enhance… │
│ (ContrastEnhance)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_40 (Add)        │ (None, 96, 96, 8) │          0 │ sobel_edge_layer… │
│                     │                   │            │ conv2d_123[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_3          │ (None, 96, 96, 8) │          0 │ gabor_conv2d_5[0… │
│ (Multiply)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_41 (Add)        │ (None, 96, 96, 8) │          0 │ contrast_enhance… │
│                     │                   │            │ add_40[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_42 (Add)        │ (None, 96, 96, 8) │          0 │ multiply_3[0][0], │
│                     │                   │            │ conv2d_123[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_124 (Conv2D) │ (None, 23, 23,    │      8,208 │ conv2d_123[0][0]  │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_132 (Conv2D) │ (None, 23, 23,    │      8,208 │ contrast_enhance… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_140 (Conv2D) │ (None, 23, 23,    │      8,208 │ add_40[0][0]      │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_148 (Conv2D) │ (None, 23, 23,    │      8,208 │ add_41[0][0]      │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_156 (Conv2D) │ (None, 23, 23,    │      8,208 │ add_42[0][0]      │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_120 (ReLU)    │ (None, 23, 23,    │          0 │ conv2d_124[0][0]  │
│                     │ 16)               │            │                 

 Total params: 93,615,393 (357.11 MB)

 Trainable params: 31,202,549 (119.03 MB)

 Non-trainable params: 7,744 (30.25 KB)

 Optimizer params: 62,405,100 (238.06 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/ops/nn.py:907: UserWarning: You are using a softmax over axis -1 of a tensor of shape (1, 5, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


Episode: 1/10, Score: 775.00
Episode: 2/10, Score: 910.40
Episode: 3/10, Score: 718.71
Episode: 4/10, Score: 870.87
Episode: 5/10, Score: 929.80
Episode: 6/10, Score: 691.95
Episode: 7/10, Score: 805.54
Episode: 8/10, Score: 816.67
